In [2]:
from selenium import webdriver
import time
import pandas as pd
import numpy as np

# 크롤링

In [3]:
import json, requests

In [6]:
festival_list = pd.read_excel("C:/Users/MINJI/MJ/2019-2/Data_Engineering/2016_festival_list.xlsx")
festival_list.head()

,연번,시도명,시군구명,축제명,시작일,종료일,축제주요내용,주최/주관 (담당자 연락처),최초개최년도 및 횟수,2016년 축제예산(안) (단위 : 백만원),...,"구,군비",기타,축제종류 (택1),축제소재,개최주기,조직형태,축제사무국 상설화여부,사업위탁 운영여부,방문객수(2015년기준) (단위 : 천명),개최장소
0,1,서울특별시,시자체,서울드럼페스티벌,2016-05-27 00:00:00,2016-05-28 00:00:00,○ 메인공연부대공연\n ○ 타악관련 전시/체험/교육행사,서울시 문화예술과\n(02-2133-2571),1999년도\n(17회),400,...,0.0,0,문화예술,"드럼, 음악",매년,공공(시),없음,부분위탁(운영),40,서울광장
1,2,서울특별시,시자체,서울거리예술축제,2016-09-28 00:00:00,2016-10-02 00:00:00,○ 국내외 초청 거리예술 공연 및 시민 참여 프로그램 등,서울시 문화예술과\n(02-2133-2573),2003년도,1600,...,0.0,0,문화예술,"예술, 거리",매년,공공(시)\n재단법인,없음,전체위탁,1000,"서울광장, 광화문광장 등"
2,3,서울특별시,시자체,서울문화의 밤,2016-08-26 00:00:00,2016-08-27 00:00:00,"○ 시민의 광장(서울광장 캠핑)\n ○ 올빼미잔치, 서울의 발견(문화탐방 프로그램) 등",서울시 문화예술과\n(02-2133-2574),2008년도\n(8회),522,...,0.0,0,문화예술,"밤, 캠핑",매년,공공(시).축제조직위\n,없음,부분위탁(운영),NaN,"서울광장, 동대문디자인플라자"
3,4,서울특별시,시자체,서울김장문화제,2016-11-04 00:00:00,2016-11-06 00:00:00,"○ 사랑의 김장나눔\n ○ 김장문화 관련 주제전시 및 체험행사, 김치 및 부재료 ...",서울시 문화예술과\n(02-2133-2570),2014년도\n(3회),1800,...,0.0,0,문화예술,"김치, 나눔, 문화",매년,공공(시군구),없음,부분위탁(운영),830,"서울광장, 광화문광장, 무교로 등"
4,5,서울특별시,시자체,국악한마당,2016-10-08 00:00:00,2016-10-11 00:00:00,"○ 메인공연(전통국악, 민요, 퓨전국악, 무용 등)\n ○ 체험 프로그램(민속놀이...",서울시 문화예술과\n(02-2133-2571),2005년도,150,...,0.0,0,전통역사,"국악, 예술",매년,공공(시군구),없음,자체추진,5,남산골 한옥마을


In [7]:
festival_list.shape

(71, 22)

# REST API 키

In [8]:
#API KEY 지정
apikey = "7f6745340f5774fddc3560cf93bf54b3" 

## 축제명으로 검색

In [9]:
url = "https://dapi.kakao.com/v2/local/search/keyword.json"
apikey = "7f6745340f5774fddc3560cf93bf54b3" 

result = list()
no_info_festival = list()

for festival in festival_list['축제명']:
    query = festival #검색을 원하는 질의어

    r = requests.get( url, params = {'query':query, 'page':45}, headers={'Authorization' : 'KakaoAK ' + apikey } ) 
    #결과를 json 형식으로 반환
    data = json.loads(r.text)
    print(festival)
    if data['meta']['total_count']!=0:        
        for i in data['documents']:
            festival_name     = festival
            address_name      = i['address_name']
            place_name        = i['place_name']
            place_url         = i['place_url']
            road_address_name = i['road_address_name']
            x                 = i['x']
            y                 = i['y']

            result.append({
                'festival_name'     : festival_name,
                'address_name'      : address_name,
                'place_name'        : place_name,
                'place_url'         : place_url,
                'road_address_name' : road_address_name,
                'x'                 : x,
                'y'                 : y
            })
    else:
        result.append({
            'festival_name'     : festival,
            'address_name'      : None,
            'place_name'        : None,
            'place_url'         : None,
            'road_address_name' : None,
            'x'                 : None,
            'y'                 : None
        })
        no_info_festival.append(festival)

서울드럼페스티벌
서울거리예술축제
서울문화의 밤
서울김장문화제
국악한마당
서울사진축제
서울아리랑페스티벌
한강 서래섬 꽃 축제
서울빛초롱축제
도시농업축제한마당
서울건축문화제
지구촌나눔한마당
서울 북 페스티벌
Seoul Book Festival
도서관아! 놀자
한강몽땅 여름축제
서울대공원 벚꽃 축제
서울대공원 장미원 축제
서울대공원 별밤 축제
서울대공원 가을 축제
점프구로축제
허준축제
겸재문화예술제
DMC 페스티벌
서울장미축제
강동선사문화축제
도심속 바다축제(안열림)
영등포 여의도 봄꽃축제
응봉산 개나리축제
태조 이성계 축제
성동디자인위크
대학로 거리공연축제
세종마을 세종주간 축제
궁중과 사대부가의
전통음식축제
육의전체험축제
윤동주문학제
인사전통문화축제
남대문 아동복축제
명동 빛 축제(년도가 달라짐)
정동야행
충무로뮤지컬영화제
북한산페스티벌
은평누리축제
성북진경축제
한양도성풍류순성(뭔지모름)
도봉구 등축제
성북다문화음식축제
서울동화축제
광나루어울마당
서울약령시 한방축제
세계거리춤축제
청룡문화제
드래곤 밸리 페스티벌
이태원 지구촌 축제
서대문독립민주축제
봉원사 서울연꽃문화축제(잘안나옴)
신촌크리스마스 거리축제
신촌물총축제
서대문 책으로 축제
금천하모니 벚꽃축제
백제고분제
석촌호수 봄꽃축제
성내천 가을음악회
석촌호수 가을음악회
책 읽는 송파 북페스티벌
한성백제문화제
석촌호수 낙엽거리축제
오금동한마음축제
관악강감찬축제
4.19혁명국민문화제
노원탈축제
태강릉·초안산궁중문화제
서리풀페스티벌


In [10]:
festival_info = pd.DataFrame(result, columns = ['festival_name','address_name', 'place_name', 'x', 'y', 'road_address_name', 'place_url'])
festival_info

,festival_name,address_name,place_name,x,y,road_address_name,place_url
0,서울드럼페스티벌,서울 중구 태평로1가 54-3,서울드럼페스티벌,126.977964394097,37.5656906233558,서울 중구 세종대로 110,http://place.map.kakao.com/7935778
1,서울거리예술축제,None,None,None,None,None,None
2,서울문화의 밤,None,None,None,None,None,None
3,서울김장문화제,None,None,None,None,None,None
4,국악한마당,경기 부천시 송내동 299-24,소리야국악한마당,126.755794838289,37.4870350969374,경기 부천시 송내대로42번길 44,http://place.map.kakao.com/383317105
5,국악한마당,전북 전주시 완산구 태평동 22-11,황금예술국악한마당,127.138860909629,35.8257533656827,전북 전주시 완산구 공북로 64,http://place.map.kakao.com/1840692408
6,서울사진축제,서울 노원구 중계동 508,서울사진축제,127.066776128739,37.6408524151384,서울 노원구 동일로 1238,http://place.map.kakao.com/27291786
7,서울아리랑페스티벌,서울 용산구 남영동 131-1,(사단)서울아리랑페스티벌조직위원회,126.973703686576,37.5407176916654,서울 용산구 한강대로72길 3,http://place.map.kakao.com/23763683
8,한강 서래섬 꽃 축제,서울 서초구 반포동 1335-1,한강서래섬 유채꽃축제,126.989929419904,37.5080426353166,,http://place.map.kakao.com/339807216
9,서울빛초롱축제,서울 중구 태평로1가 1,서울빛초롱축제,126.977782252568,37.5691900568791,,http://place.map.kakao.com/187995053


In [11]:
len(no_info_festival)

41

In [12]:
festival_info.to_csv("crawling_festival_info_1.csv", encoding='CP949', index=False)

## 크롤링 된 축제정보를 보정한 데이터 불러오기

In [31]:
festival_info_2 = pd.read_csv("crawling_festival_info_2.csv", encoding='CP949')

In [32]:
festival_info_2

,festival_name,address_name,place_name,x,y,road_address_name,place_url
0,서울드럼페스티벌,서울 중구 태평로1가 54-3,서울드럼페스티벌,126.977964,37.565691,서울 중구 세종대로 110,http://place.map.kakao.com/7935778
1,서울거리예술축제,NaN,NaN,NaN,NaN,NaN,NaN
2,서울문화의 밤,NaN,NaN,NaN,NaN,NaN,NaN
3,서울김장문화제,NaN,NaN,NaN,NaN,NaN,NaN
4,국악한마당,NaN,NaN,NaN,NaN,NaN,NaN
5,서울사진축제,서울 노원구 중계동 508,서울사진축제,127.066776,37.640852,서울 노원구 동일로 1238,http://place.map.kakao.com/27291786
6,서울아리랑페스티벌,NaN,NaN,NaN,NaN,NaN,NaN
7,한강 서래섬 꽃 축제,서울 서초구 반포동 1335-1,한강서래섬 유채꽃축제,126.989929,37.508043,NaN,http://place.map.kakao.com/339807216
8,서울빛초롱축제,서울 중구 태평로1가 1,서울빛초롱축제,126.977782,37.569190,NaN,http://place.map.kakao.com/187995053
9,도시농업축제한마당,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
festival_info_2[['x', 'y']].dropna().shape

(25, 2)

In [34]:
festival_info_3 = pd.merge(festival_info_2, festival_list[['축제명', '시작일', '종료일']], left_on='festival_name', right_on='축제명')
festival_info_3.head()

,festival_name,address_name,place_name,x,y,road_address_name,place_url,축제명,시작일,종료일
0,서울드럼페스티벌,서울 중구 태평로1가 54-3,서울드럼페스티벌,126.977964,37.565691,서울 중구 세종대로 110,http://place.map.kakao.com/7935778,서울드럼페스티벌,2016-05-27 00:00:00,2016-05-28 00:00:00
1,서울거리예술축제,NaN,NaN,NaN,NaN,NaN,NaN,서울거리예술축제,2016-09-28 00:00:00,2016-10-02 00:00:00
2,서울문화의 밤,NaN,NaN,NaN,NaN,NaN,NaN,서울문화의 밤,2016-08-26 00:00:00,2016-08-27 00:00:00
3,서울김장문화제,NaN,NaN,NaN,NaN,NaN,NaN,서울김장문화제,2016-11-04 00:00:00,2016-11-06 00:00:00
4,국악한마당,NaN,NaN,NaN,NaN,NaN,NaN,국악한마당,2016-10-08 00:00:00,2016-10-11 00:00:00


In [35]:
festival_info_3['시작일'] = festival_info_3['시작일'].dt.strftime('%Y-%m-%d')
festival_info_3['종료일'] = festival_info_3['종료일'].dt.strftime('%Y-%m-%d')

In [37]:
festival_info_3[['festival_name', 'x', 'y', 'address_name', '시작일', '종료일']].dropna().reset_index(drop=True)

,festival_name,x,y,address_name,시작일,종료일
0,서울드럼페스티벌,126.977964,37.565691,서울 중구 태평로1가 54-3,2016-05-27,2016-05-28
1,서울사진축제,127.066776,37.640852,서울 노원구 중계동 508,2016-11-01,2016-11-03
2,한강 서래섬 꽃 축제,126.989929,37.508043,서울 서초구 반포동 1335-1,2016-05-14,2016-05-15
3,서울빛초롱축제,126.977782,37.569190,서울 중구 태평로1가 1,2016-11-04,2016-11-20
4,서울 북 페스티벌\nSeoul Book Festival\n도서관아! 놀자,37.566295,126.977945,"서울 중구 세종대로 110 (태평로1가, 서울시청)",2016-09-02,2016-09-03
5,한강몽땅 여름축제,127.041625,37.538944,서울 성동구 성수동1가 642-25,2016-07-15,2016-08-21
6,서울대공원 벚꽃 축제,127.019907,37.426792,경기 과천시 막계동 658-2,2016-04-09,2016-04-24
7,서울대공원 장미원 축제,127.020038,37.426770,경기 과천시 막계동 658-2,2016-05-28,2016-06-12
8,허준축제,126.852247,37.568210,서울 강서구 가양동 1471,2016-10-07,2016-10-09
9,DMC 페스티벌,126.891267,37.580415,서울 마포구 상암동 1604,2016-10-01,2016-10-11
